In [ ]:
import tableauserverclient as TSC
import pandas as pd
import json
from pandas import json_normalize

: 

In [4]:
#Connect to Tableau Online
tableau_auth = TSC.PersonalAccessTokenAuth('python', 
                                           'FZhLgGFrT2moQnr86t0GuQ==:FvA0OyA51ayOOdA5Tfc36cbclDhjM4Oc', 
                                           'globalizationpartners')
server = TSC.Server('https://us-east-1.online.tableau.com', use_server_version=True)

In [5]:
df_views=pd.DataFrame()
with server.auth.sign_in(tableau_auth):
    req_option_views = TSC.RequestOptions(pagesize=1000)
    all_views, pagination_item = server.views.get(req_options=req_option_views, usage=True)
    df_views['View Name']=[view.name for view in all_views]
    df_views['Total Views']=[view.total_views for view in all_views]
    df_views['Workbook_id']=[view.workbook_id for view in all_views]
df_views.head()        

,View Name,Total Views,Workbook_id
0,Missing PL Accounts from MDM,35,866f9120-e0dd-4b3b-8466-21c52f10ff4f
1,Missing BS Accounts from MDM,6,866f9120-e0dd-4b3b-8466-21c52f10ff4f
2,Overview,41,9f6b0b6b-c228-4918-b53e-1e63b97ae07c
3,User Role Drilldown,20,9f6b0b6b-c228-4918-b53e-1e63b97ae07c
4,Login Activity Drilldown,20,9f6b0b6b-c228-4918-b53e-1e63b97ae07c


In [6]:
df_views_wk=pd.DataFrame(df_views.groupby('Workbook_id')['Total Views'].sum()).reset_index()
df_views_wk.head()

,Workbook_id,Total Views
0,01f42c4d-5d0c-4918-ac6e-968d6406d535,66
1,02960879-6e83-4fab-ae35-e3291d42e595,19
2,02ce1524-4c2d-452b-a552-7c9a3e26b957,169
3,0466c8e0-0860-4bab-af5f-b5e6704409f6,169
4,04bef754-739a-4882-913f-8ebc4cdc97dd,44


In [7]:
df_projects=pd.DataFrame()
with server.auth.sign_in(tableau_auth):
        req_option_projects = TSC.RequestOptions(pagesize=1000)
        all_projects, pagination_item = server.projects.get(req_options=req_option_projects)
        df_projects['Name']=[proj.name for proj in all_projects]
        df_projects['Id']=[proj.id for proj in all_projects]
        df_projects['Parent_id']=[proj.parent_id for proj in all_projects]
df_projects.head()

,Name,Id,Parent_id
0,default,98639924-86e6-4518-8396-ac78943834ba,None
1,Executive Dashboards,735b5eae-9a07-44dd-a86d-f2a7538f10ed,1e53aadb-27b5-4432-ad08-6f44593bde19
2,Financial - Expenses,77b556fd-2583-4375-aeea-5023651fec8b,41be2a72-9f60-499c-a931-8576cb604c2b
3,People Resources,e9186f23-853d-4295-a540-a3dcdecfea21,None
4,Corporate Ad Hoc,7047054b-3c70-4c99-a400-902903a2e335,1e53aadb-27b5-4432-ad08-6f44593bde19


In [8]:
df_pr=df_projects.copy()
i=df_pr.iloc[:,-1].isna().sum()
j=df_projects.shape[0]
while i<j:
    df_pr=df_pr.merge(df_projects,how='left',
                                  left_on=df_pr.iloc[:,-1].name,right_on='Id')
    i=df_pr.iloc[:,-1].isna().sum()

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Name_x', 'Id_x', 'Parent_id_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


In [9]:
df_pr['Global Folder']=[df_pr.iloc[idx, col] for idx, col in enumerate(df_pr.shape[1]-df_pr.isna().sum(axis=1)-1)]
df_projects_groups=df_pr.iloc[:,[1,-1]]
df_projects_groups.head()

,Id_x,Global Folder
0,98639924-86e6-4518-8396-ac78943834ba,98639924-86e6-4518-8396-ac78943834ba
1,735b5eae-9a07-44dd-a86d-f2a7538f10ed,1e53aadb-27b5-4432-ad08-6f44593bde19
2,77b556fd-2583-4375-aeea-5023651fec8b,6ad47374-16bd-486e-80a2-d41fa59d4390
3,e9186f23-853d-4295-a540-a3dcdecfea21,e9186f23-853d-4295-a540-a3dcdecfea21
4,7047054b-3c70-4c99-a400-902903a2e335,1e53aadb-27b5-4432-ad08-6f44593bde19


In [10]:
df_workbooks=pd.DataFrame()
with server.auth.sign_in(tableau_auth):
    req_option = TSC.RequestOptions(pagesize=1000)
    all_workbooks, pag_item = server.workbooks.get(req_options=req_option)
    i=0
    for wk in all_workbooks:
        server.workbooks.populate_connections(wk)
        conns = wk.connections
        user = server.users.get_by_id(wk.owner_id)
        for conn in conns:
            df_workbooks[i]=[wk.name,wk.id,wk.project_name,wk.project_id,user.name,conn.datasource_id,conn.datasource_name,conn.connection_type]
            i+=1

/var/folders/9n/96h_c8kn7z927k6pw026_0xr0000gn/T/ipykernel_46841/321191349.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_workbooks[i]=[wk.name,wk.id,wk.project_name,wk.project_id,user.name,conn.datasource_id,conn.datasource_name,conn.connection_type]


In [11]:
df_workbooks=df_workbooks.T
df_workbooks.columns=['Wk Name','Wk Id','Project Name','Project Id','Owner','Datasource Id','Datsource Name','Connection Type']
df_workbooks.head(50)

,Wk Name,Wk Id,Project Name,Project Id,Owner,Datasource Id,Datsource Name,Connection Type
0,Acumatica GL,cb4872fc-5ccb-4efb-bd05-a53a652e8a2c,Financial - Expenses,77b556fd-2583-4375-aeea-5023651fec8b,acarsi@g-p.com,18462f43-3bd0-45de-bad5-dfcc4229ac0f,Acumatica GL,sqlproxy
1,Missing PL Accounts from MDM,866f9120-e0dd-4b3b-8466-21c52f10ff4f,Information Tools,f9a70faa-391e-489a-a5a2-5c1f3134f519,acarsi@g-p.com,101d0bbf-3451-414b-b021-05e9225c523a,Missing PL Accounts from MDM,postgres
2,Missing PL Accounts from MDM,866f9120-e0dd-4b3b-8466-21c52f10ff4f,Information Tools,f9a70faa-391e-489a-a5a2-5c1f3134f519,acarsi@g-p.com,6784c72c-d592-4a36-ba4c-ebea4913186c,Missing BS Accounts from MDM,postgres
3,Admin Insights Starter,9f6b0b6b-c228-4918-b53e-1e63b97ae07c,Information Tools,f9a70faa-391e-489a-a5a2-5c1f3134f519,acarsi@g-p.com,7be5f72c-c9d5-418a-a04e-9c0a2d62bb3e,TS Events,sqlproxy
4,Admin Insights Starter,9f6b0b6b-c228-4918-b53e-1e63b97ae07c,Information Tools,f9a70faa-391e-489a-a5a2-5c1f3134f519,acarsi@g-p.com,8e255c37-1c53-4795-adb4-e69447cea404,TS Users,sqlproxy
5,Countries Missing from Regions Hierarchy,216ede8c-b1a9-4940-aab1-70830249ceb8,Information Tools,f9a70faa-391e-489a-a5a2-5c1f3134f519,acarsi@g-p.com,bf243a87-1332-4161-a6df-2871f104a930,missing_countries,postgres
6,Information Catalog,8aca0cbd-c6fb-45e1-a2d3-57b5d0f14c46,Information Tools,f9a70faa-391e-489a-a5a2-5c1f3134f519,acarsi@g-p.com,5016afe8-e539-4530-97f0-08ae9462c8f0,info_schema,postgres
7,Information Catalog,8aca0cbd-c6fb-45e1-a2d3-57b5d0f14c46,Information Tools,f9a70faa-391e-489a-a5a2-5c1f3134f519,acarsi@g-p.com,58ba9bec-b179-4e08-b6b4-e38e79cce942,sample_values,postgres
8,Accounting Commissions Dashboard,d450f98e-02bd-4745-af68-227c3773c4b6,Finance Legacy,24f71657-42c2-496b-839b-c78178ad1e2a,acarsi@g-p.com,d3ac61e4-1a39-490e-9a5f-c87aeb4bb003,v_headcount_dashboard (repo.v_headcount_dashbo...,postgres
9,Missing Departments from MDM,2d0d9994-9b66-438e-8ec0-5f3afec82569,Information Tools,f9a70faa-391e-489a-a5a2-5c1f3134f519,acarsi@g-p.com,90996b41-c1e2-4dc8-b623-cfae25d3b899,Custom SQL Query (gpbidb),postgres


In [12]:
df_workbooks.insert(6, 'Total views',df_workbooks['Wk Id'].map(df_views_wk.set_index('Workbook_id')['Total Views']))
df_workbooks.insert(6, 'Main Folder Id',df_workbooks['Project Id'].map(df_projects_groups.set_index('Id_x')['Global Folder']))
df_workbooks.insert(6, 'Main Folder Name',df_workbooks['Main Folder Id'].map(df_projects.set_index('Id')['Name']))

In [13]:
df_workbooks.head()

,Wk Name,Wk Id,Project Name,Project Id,Owner,Datasource Id,Main Folder Name,Main Folder Id,Total views,Datsource Name,Connection Type
0,Acumatica GL,cb4872fc-5ccb-4efb-bd05-a53a652e8a2c,Financial - Expenses,77b556fd-2583-4375-aeea-5023651fec8b,acarsi@g-p.com,18462f43-3bd0-45de-bad5-dfcc4229ac0f,Tableau Admin,6ad47374-16bd-486e-80a2-d41fa59d4390,619,Acumatica GL,sqlproxy
1,Missing PL Accounts from MDM,866f9120-e0dd-4b3b-8466-21c52f10ff4f,Information Tools,f9a70faa-391e-489a-a5a2-5c1f3134f519,acarsi@g-p.com,101d0bbf-3451-414b-b021-05e9225c523a,Tableau Admin,6ad47374-16bd-486e-80a2-d41fa59d4390,41,Missing PL Accounts from MDM,postgres
2,Missing PL Accounts from MDM,866f9120-e0dd-4b3b-8466-21c52f10ff4f,Information Tools,f9a70faa-391e-489a-a5a2-5c1f3134f519,acarsi@g-p.com,6784c72c-d592-4a36-ba4c-ebea4913186c,Tableau Admin,6ad47374-16bd-486e-80a2-d41fa59d4390,41,Missing BS Accounts from MDM,postgres
3,Admin Insights Starter,9f6b0b6b-c228-4918-b53e-1e63b97ae07c,Information Tools,f9a70faa-391e-489a-a5a2-5c1f3134f519,acarsi@g-p.com,7be5f72c-c9d5-418a-a04e-9c0a2d62bb3e,Tableau Admin,6ad47374-16bd-486e-80a2-d41fa59d4390,132,TS Events,sqlproxy
4,Admin Insights Starter,9f6b0b6b-c228-4918-b53e-1e63b97ae07c,Information Tools,f9a70faa-391e-489a-a5a2-5c1f3134f519,acarsi@g-p.com,8e255c37-1c53-4795-adb4-e69447cea404,Tableau Admin,6ad47374-16bd-486e-80a2-d41fa59d4390,132,TS Users,sqlproxy


In [14]:
df=df_workbooks.sort_values(['Main Folder Name','Total views'],ascending=[True,False]).groupby('Main Folder Name').head(5)
df.head()

,Wk Name,Wk Id,Project Name,Project Id,Owner,Datasource Id,Main Folder Name,Main Folder Id,Total views,Datsource Name,Connection Type
268,Brand Tracker,3127476f-2c54-4c1d-b4eb-f6d8ab41e415,Brand,7995a52e-1753-40ce-a533-e2744afa3c63,operez@g-p.com,b5420d14-05d2-4503-9b78-d2d7cff7c24c,Brand,7995a52e-1753-40ce-a533-e2744afa3c63,498,Brand - Share of Voice Data,sqlproxy
269,Brand Tracker,3127476f-2c54-4c1d-b4eb-f6d8ab41e415,Brand,7995a52e-1753-40ce-a533-e2744afa3c63,operez@g-p.com,a9562b15-28df-4bed-b7f3-81108ead36cd,Brand,7995a52e-1753-40ce-a533-e2744afa3c63,498,Brand - SEO Data,sqlproxy
270,Brand Tracker,3127476f-2c54-4c1d-b4eb-f6d8ab41e415,Brand,7995a52e-1753-40ce-a533-e2744afa3c63,operez@g-p.com,c3e6c67a-5344-4d2d-a676-7ab6863eba2b,Brand,7995a52e-1753-40ce-a533-e2744afa3c63,498,Brand - Radio Ads Data,sqlproxy
271,Brand Tracker,3127476f-2c54-4c1d-b4eb-f6d8ab41e415,Brand,7995a52e-1753-40ce-a533-e2744afa3c63,operez@g-p.com,447f264a-bb9d-42a6-99c4-91033643f77d,Brand,7995a52e-1753-40ce-a533-e2744afa3c63,498,Brand - Social Media Data,sqlproxy
272,Brand Tracker,3127476f-2c54-4c1d-b4eb-f6d8ab41e415,Brand,7995a52e-1753-40ce-a533-e2744afa3c63,operez@g-p.com,2737ec12-5c31-4660-847c-81fd52a94acc,Brand,7995a52e-1753-40ce-a533-e2744afa3c63,498,Brand - Social Listening Data,sqlproxy


In [15]:
with pd.ExcelWriter('/Users/ivanmartinez/Tableau_Content_Summary.xlsx') as writer:
...     df_workbooks.to_excel(writer, sheet_name='All',index=False)
...     df.to_excel(writer, sheet_name='Top Dashboards',index=False)

### MetaData

In [ ]:
with server.auth.sign_in(tableau_auth):
    query = """
          query datasources{
             datasources {
                        id
                        name
                        fields{
                            name
                             ...on CalculatedField {
                             formula
                            }
                        }
                    }
                }
                            """
    resp_datasources = server.metadata.query(query)

In [ ]:
print(resp_datasources['data']['datasources'])

In [ ]:
df_datasources=json_normalize(resp_datasources['data']['datasources'], record_path =['fields'],meta=['id','name'],
                             meta_prefix='datasource_') 
df_datasources.head(15)

In [ ]:
with server.auth.sign_in(tableau_auth):
    query = """
          query tables{
            customSQLTables(filter: {name: "Polaris KPIs - Phase 1"}){
                        id
                        name
                        query
                    }
                }
                            """
    resp_customtable = server.metadata.query(query)

In [ ]:
print(resp_customtable)

In [ ]:
df_customtable=json_normalize(resp_customtable['data']['customSQLTables']) 
df_customtable.head()

In [ ]:
with server.auth.sign_in(tableau_auth):
    query = """
          query tables{
            databaseTables{
                        id
                        name
                        schema

                    }
                }
                            """
    resp_tables = server.metadata.query(query)

In [ ]:
print(resp_tables)

In [ ]:
df_tables=json_normalize(resp_tables['data']['databaseTables']) 
df_tables.head()

In [ ]:
server.auth.sign_out()

In [ ]:
with pd.ExcelWriter('/Users/ivanmartinez/Tableau_Content_Summary.xlsx') as writer:
...     df_workbooks.to_excel(writer, sheet_name='All',index=False)
...     df.to_excel(writer, sheet_name='Top Dashboards',index=False)
...     df_datasources.to_excel(writer, sheet_name='Fields',index=False)
...     df_customtable.to_excel(writer, sheet_name='Queries',index=False)
...     df_tables.to_excel(writer, sheet_name='PostgreSQL Tables',index=False)

### END

In [ ]:
with server.auth.sign_in(tableau_auth):
    query = """
          query datasources($name: String){
             datasources(filter: {name: $name}) {
                        id
                        name
                        fields{
                            name
                             ...on CalculatedField {
                             formula
                            }
                        }
                    }
                }
                            """
    resp = server.metadata.query(query,{"name":"Current Active Customers"})
    print(resp['data'])

In [ ]:
with server.auth.sign_in(tableau_auth):
    query = """
          query tables{
            embeddedDatasources{
                        id
                        name
                        workbook{
                            name
                        }
                        fields{
                            name
                        }
                    }
                }
                            """
    resp = server.metadata.query(query)
    print(resp)

In [ ]:
#databases= othr formats (xlsx....)
with server.auth.sign_in(tableau_auth):
    query = """
            {
                
                   databases{
                       name
                       
                   }
                            }
                            """
    resp = server.metadata.query(query)
    print(resp['data'])